In [ ]:
#imports

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/Users/alex/Organoids/Code/ProcessingPack_Organoids/processingpack')

from copy import deepcopy
from pathlib import Path

import cv2
import numpy as np
import skimage
from skimage import measure
import pandas as pd
import matplotlib.pyplot as pl

import ast
import operator

import org_extension
import chip
import experiment as exp
import chipcollections as collections
from org_extension import *

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
#use fiji to to open stitched image from first timepoint
#hover cursor over center of well for top left, top right, bottom left, and bottom right corners of complete array

oc = ((1002,3548), (7768,2951), (1821,12696), (8624,12114))

In [ ]:
#set parameters for processing each well
from datetime import datetime

#number of complete subarrays in stitched image
#typically it's 3 columns and somewhere between 3-5 rows depending on image
subarray_dims = (3,4) #col,row

#change tile dims for 100um or 200um microwells
#100um
tile_dims = (20, 20)
#200um
# tile_dims = (10, 10)

#leave these alone, don't actually impact processing
channel = '2bf'
exposure = 10

#change to reflect which well you're processing
#well A1 = 1, B1 = 5, etc. 
well_id = 'B2'
well =6

#change to reflect date of processing
date = datetime(2020, 11, 24)


In [ ]:
#use terminal and text editor with find and replace to get list of all timepoint from timecourse
for timepoint in ('20200710-193019', '20200710-214217', '20200710-235402', '20200711-020548', '20200711-041733', '20200711-062910', '20200711-084041', '20200711-105217', '20200711-130359', '20200711-151541', '20200711-172720', '20200711-193857', '20200711-215029', '20200712-000204', '20200712-021343', '20200712-042520', '20200712-063643', '20200712-084815', '20200712-105948', '20200712-131121', '20200712-152253', '20200712-173425', '20200712-194557', '20200712-215730', '20200713-000900', '20200713-022030', '20200713-043202', '20200713-064330', '20200713-085502', '20200713-110630', '20200713-131804', '20200713-152925', '20200713-174053', '20200713-195223', '20200713-220354', '20200714-001523', '20200714-022646', '20200714-043811', '20200714-064939', '20200714-090103', '20200714-111230', '20200714-132358', '20200714-153529', '20200714-174657', '20200714-195830', '20200714-220959', '20200715-002133', '20200715-023306', '20200715-044439', '20200715-065610', '20200715-090740', '20200715-111910', '20200715-132930', '20200715-153949', '20200715-175108', '20200715-200244', '20200715-221421', '20200716-002555', '20200716-023729', '20200716-044907', '20200716-070034', '20200716-091153'):
    #change srcHandle and targetHandle to match file location for stitched images
    srcHandle = Path('/Volumes/Backup Plus/raw_imaging_data/Nikon/4230_P53_ARID1A_200um_20200710/{0}/{1}-{0}_timecourse/6Mcherry/StitchedImages/StitchedImg_200_6Mcherry_0.tif'.format(well_id, timepoint))
    targetHandle = Path('/Volumes/Backup Plus/raw_imaging_data/Nikon/4230_P53_ARID1A_200um_20200710/{0}/{1}-{0}_timecourse/6Mcherry/StitchedImages/RotatedImg_200_6Mcherry_0.tif'.format(well_id, timepoint))
    rot_img = readAndRotateImg(srcHandle, oc, subarray_dims, targetHandle = targetHandle)
    rot_corners = transformCorners(readTiff(srcHandle), rot_img, oc)
    divisions = getPartitions(rot_corners, subarray_dims)
    #change root  and outRoot below to match desired location for processed data
    root = '/Volumes/Rapid_lbx/processed_data/Nikon/4230_P53_ARID1A_200um_20200710/{0}/mCherry/'.format(well_id)
    description = 'timepoint_{0}'.format(timepoint)
    operator = 'AS'
    #change stampWidth depending on 100um vs 200um microwells
    #100um
    chip.ChipImage.stampWidth = 115
    #200um
#     chip.ChipImage.stampWidth = 225
    e = exp.Experiment(description, root, operator) #establish experiment
    #change pinlist path to location of ProcessingPack_Organoids
    #100um
    pinlist_path = '/Users/alex/Organoids/Code/ProcessingPack_Organoids/Pinlist/20x20_pinlist.csv'
    #200um
#     pinlist_path = '/Users/alex/Organoids/Code/ProcessingPack_Organoids/Pinlist/10x10_pinlist.csv'    
    pinlist = e.read_pinlist(pinlist_path)#establish pinlist
    arrayRepo = processTiles(str(targetHandle), e, pinlist, divisions, tile_dims, subarray_dims, channel, exposure, date = date, well = well, desc = e.info)
    outRoot = Path('/Volumes/Rapid_lbx/processed_data/Nikon/4230_P53_ARID1A_200um_20200710/{0}/mCherry'.format(well_id)) #Folder to write summary image and CSV

    ar = arrayRepo[0, 0] #Need any sub-array to generate the hash, so the first will do
    hashStr = generateWellIdentifier(ar)
    summary_imgPath = outRoot.joinpath('{}.tif'.format(hashStr))
    summary_csvPath = outRoot.joinpath('{}.csv'.format(hashStr))
    
    summaryDF = summarizeArrayRepo(arrayRepo)
    summaryDF.to_csv(summary_csvPath)
    writeSubArraySummaryImg(arrayRepo, summary_imgPath)